In [6]:
import torch
import torch.nn.functional as F

from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

In [7]:
class MLP(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(MLP, self).__init__()

        self.lins = torch.nn.ModuleList()
        self.lins.append(torch.nn.Linear(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.lins.append(torch.nn.Linear(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.lins.append(torch.nn.Linear(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for lin in self.lins:
            lin.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x):
        for i, lin in enumerate(self.lins[:-1]):
            x = lin(x)
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lins[-1](x)
        return torch.log_softmax(x, dim=-1)


In [8]:
def train(model, x, y_true, train_idx, optimizer):
    model.train()

    optimizer.zero_grad()
    out = model(x[train_idx])
    loss = F.nll_loss(out, y_true.squeeze(1)[train_idx])
    loss.backward()
    optimizer.step()

    return loss.item()

In [9]:
@torch.no_grad()
def test(model, x, y_true, split_idx, evaluator):
    model.eval()

    out = model(x)
    y_pred = out.argmax(dim=-1, keepdim=True)

    train_acc = evaluator.eval({
        'y_true': y_true[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': y_true[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': y_true[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['acc']

    return train_acc, valid_acc, test_acc

In [10]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)

dataset = PygNodePropPredDataset(name='ogbn-arxiv', root='../dataset')
split_idx = dataset.get_idx_split()
data = dataset[0]

In [14]:
x = data.x
x = x.to(device)

In [17]:
y_true = data.y.to(device)
train_idx = split_idx['train'].to(device)

In [18]:
train_idx.shape

torch.Size([90941])

In [19]:
model = MLP(x.size(-1), 256, dataset.num_classes, 3, 0.5).to(device)

In [21]:
evaluator = Evaluator(name='ogbn-arxiv')

In [22]:
print(evaluator.expected_input_format) 

==== Expected input format of Evaluator for ogbn-arxiv
{'y_true': y_true, 'y_pred': y_pred}
- y_true: numpy ndarray or torch tensor of shape (num_node, num_task)
- y_pred: numpy ndarray or torch tensor of shape (num_node, num_task)
where y_pred stores predicted class label (integer),
num_task is 1, and each row corresponds to one node.



In [26]:
for run in range(10):
    model.reset_parameters()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    for epoch in range(1, 1 + 500):
        loss = train(model, x, y_true, train_idx, optimizer)
        result = test(model, x, y_true, split_idx, evaluator)

        if epoch % 50 == 0:
            train_acc, valid_acc, test_acc = result
            print(f'Run: {run + 1:02d}, '
                  f'Epoch: {epoch:02d}, '
                  f'Loss: {loss:.4f}, '
                  f'Train: {100 * train_acc:.2f}%, '
                  f'Valid: {100 * valid_acc:.2f}%, '
                  f'Test: {100 * test_acc:.2f}%')

Run: 01, Epoch: 50, Loss: 1.6400, Train: 55.36%, Valid: 55.23%, Test: 53.00%
Run: 01, Epoch: 100, Loss: 1.5435, Train: 58.76%, Valid: 56.30%, Test: 54.02%
Run: 01, Epoch: 150, Loss: 1.4916, Train: 60.67%, Valid: 56.97%, Test: 54.56%
Run: 01, Epoch: 200, Loss: 1.4680, Train: 61.85%, Valid: 57.35%, Test: 55.06%
Run: 01, Epoch: 250, Loss: 1.4457, Train: 62.67%, Valid: 57.55%, Test: 55.05%
Run: 01, Epoch: 300, Loss: 1.4318, Train: 63.18%, Valid: 57.61%, Test: 55.30%
Run: 01, Epoch: 350, Loss: 1.4197, Train: 63.61%, Valid: 57.79%, Test: 55.69%
Run: 01, Epoch: 400, Loss: 1.4183, Train: 63.90%, Valid: 58.05%, Test: 55.77%
Run: 01, Epoch: 450, Loss: 1.4122, Train: 64.07%, Valid: 57.85%, Test: 55.62%
Run: 01, Epoch: 500, Loss: 1.4088, Train: 64.39%, Valid: 57.80%, Test: 55.81%
Run: 02, Epoch: 50, Loss: 1.6403, Train: 55.02%, Valid: 54.78%, Test: 52.69%
Run: 02, Epoch: 100, Loss: 1.5402, Train: 58.81%, Valid: 56.57%, Test: 54.34%
Run: 02, Epoch: 150, Loss: 1.4869, Train: 60.75%, Valid: 57.03%, T

KeyboardInterrupt: 

In [28]:
x[train_idx].shape

torch.Size([90941, 128])